In [ ]:
import tkinter as tk
from tkinter import ttk
import cv2
import numpy as np
from PIL import Image, ImageTk
import pyautogui
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
# from face_mediapipe import FaceTracker
from Main_model.showframeVisualization import FrameShow_head_face
import time

import random
import os
from datetime import datetime
import csv
# import screeninfo

class Toggle:
    def __init__(self, initial_state=False):
        self.state = initial_state
    
    def toggle(self):
        self.state = not self.state
        return self.state
    
    def __bool__(self):
        return self.state
    
class EyeHeadTrackingApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Eye & head Tracking Application")

        self.tracker = FrameShow_head_face(
            model_path='Main_model/face_landmarker.task',
            isHeadposeOn = True, 
            isFaceOn = True
        )
        self.timestamp_ms = 0

        # self.output_dir = "model/output_from_capture"
        self.output_dir = "Data_collected"
        os.makedirs(self.output_dir, exist_ok=True)
        self.file_counter = 1
        self.update_next_available_number()

        # Get screen resolution before initializing other components
        screen_size = pyautogui.size()
        self.screen_width = screen_size[0]
        self.screen_height = screen_size[1]
        self.webcam_width = 640
        self.webcam_height = 360
        # self.original_canvas_width = 1728
        # self.original_canvas_height = 1117
        
        self.original_canvas_width = screen_size[0] # 1708
        self.original_canvas_height = screen_size[1] # 948

        # UI state flags
        # self.preview_active = False
        self.capture_in_progress = False

        # self.switch_parameterOnScreen = False

        self.position_left_label = None
        self.position_right_label = None
        self.gaze_label = None
        self.canvas_normal_state = True
        self.controls_visible = True
        self.focal_length = 1000
        self.FACE_WIDTH_CM = 14
        self.current_dot = None
        self.capture_counter = 0
        # self.preview_var = tk.BooleanVar(value=False)
        # self.parameter_var = tk.BooleanVar(value=False)

        # Initialize parameters
        self.current_parameters = {
            'pich': None,
            'yaw': None,
            'roll': None,
            
            'face_min_position': None,
            'face_max_position': None,
            'face_size_width': None,
            'face_size_height': None,
            'face_center_position': None,

            'nose_position': None,
            'mouth_left_position': None,
            'mouth_right_position': None,
            'chin_position': None,
            'cheek_left_position': None,
            'cheek_right_position': None,

            'left_eye_box': (None, None),
            'right_eye_box': (None, None),
            'left_eye_state': None,
            'right_eye_state': None,
            'left_eye_ear': None,
            'right_eye_ear': None,
            'left_eye_position': (None, None),
            'right_eye_position': (None, None),
            'eye_socket_right_center': (None, None),
            'eye_socket_left_center': (None, None),
            'center_between_eye_left_and_right': (None, None),

            'dot_position_x': None,
            'dot_position_y': None,
            'webcam_resolution_width': None,
            'webcam_resolution_height': None,
            'screen_resolution_width': self.screen_width,
            'screen_resolution_height': self.screen_height,
            'preview_width': None,
            'preview_height': None,
            'canvas_width': None,
            'canvas_height': None,

            'gaze_direction': None,
            'posture': None,

            'distance': None,
            'distance_cm_from_eye': None,
            'chin_depth': None
        }

        self.setup_gui()

        # Initialize webcam
        # self.cap = cv2.VideoCapture(1)
        self.cap = self.tracker.create_webcam_capture(1)  # Use 1 for external webcam
        # self.tracker.set_Label_display(False)
        # 1920x1080 FHD
        # 1280x720 HD
        min_width = 1280
        min_height = 720

        # Get the maximum resolution supported by the camera
        max_width = int(self.cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        max_height = int(self.cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

        self.cap.set(cv2.CAP_PROP_FRAME_WIDTH, max(max_width, min_width))
        self.cap.set(cv2.CAP_PROP_FRAME_HEIGHT, max(max_height, min_height))

        # Start preview update
        self.update_preview()
        
    def setup_gui(self):
        """Main setup function that calls all UI component setups"""
        self.isMaskOn = Toggle(False)
        self.isDrawVisualization = Toggle(False)
        self.isDrawBox = Toggle(False)
        self.isShowParameter = Toggle(False)
        self.isShowCamera = Toggle(False)
        self.bar_visible = Toggle(True)
        self.current_calibration_index = 0
        self.isShowLabelOnDot = Toggle(False)
        
        self.setup_main_structure()
        
        # Initialize toggle function before setting up top bar
        # self.toggle_bar = lambda: self.toggle_bar_visibility()

        self.setup_top_bar()
        self.setup_status_section()
        self.setup_status_section_for_set_random()
        self.setup_main_content()
        self.setup_preview_section()
        self.setup_parameters_section()
        self.setup_countdown_labels()
# ---------------------------------------------------------
    def setup_status_section(self):
        """Setup the status section with better layout"""
        if not hasattr(self, 'top_bar'):
            return

        # Create status container in the top bar
        self.status_container = tk.Frame(self.top_bar, bg='#2C3E50')
        self.status_container.pack(side=tk.RIGHT, padx=2)

        # Left status frame for distance and posture
        self.status_left = tk.Frame(self.status_container, bg='#2C3E50')
        self.status_left.pack(side=tk.LEFT, padx=2)

        # Distance label
        self.distance_label = tk.Label(
            self.status_left,
            text="Distance: -- cm",
            font=('Arial', 11),
            bg='#2C3E50',
            fg='white',
            width=15,  # Fixed width
            anchor='w'  # Left align text
        )
        self.distance_label.pack(pady=2)

        # Posture label
        self.posture_label = tk.Label(
            self.status_left,
            text="Posture: --",
            font=('Arial', 11),
            bg='#2C3E50',
            fg='white',
            width=15,  # Fixed width
            anchor='w'  # Left align text
        )
        self.posture_label.pack(pady=2)

        # Center status frame for eye positions
        self.status_center = tk.Frame(self.status_container, bg='#2C3E50')
        self.status_center.pack(side=tk.LEFT, padx=2)

        # Left eye position
        self.position_left_label = tk.Label(
            self.status_center,
            text="Left: (--, --)",
            font=('Arial', 11),
            bg='#2C3E50',
            fg='white',
            width=15,  # Fixed width
            anchor='w'  # Left align text
        )
        self.position_left_label.pack(pady=2)

        # Right eye position
        self.position_right_label = tk.Label(
            self.status_center,
            text="Right: (--, --)",
            font=('Arial', 11),
            bg='#2C3E50',
            fg='white',
            width=15,  # Fixed width
            anchor='w'  # Left align text
        )
        self.position_right_label.pack(pady=2)

        # Right status frame for gaze and countdown
        self.status_right = tk.Frame(self.status_container, bg='#2C3E50')
        self.status_right.pack(side=tk.LEFT, padx=2)

        # Gaze direction
        self.gaze_label = tk.Label(
            self.status_right,
            text="Gaze: --",
            font=('Arial', 11),
            bg='#2C3E50',
            fg='white',
            width=15,  # Fixed width
            anchor='w'  # Left align text
        )
        self.gaze_label.pack(pady=2)

        # Countdown label
        self.countdown_label = tk.Label(
            self.status_right,
            text="Countdown",
            font=('Arial', 12),
            bg='#2C3E50',
            fg='white',
            width=10,  # Fixed width
            anchor='w'  # Left align text
        )
        self.countdown_label.pack(pady=2)

        # # Frame for signal label (added container frame)
        # self.signal_frame = tk.Frame(self.status_container, bg='#2C3E50')
        # self.signal_frame.pack(side=tk.LEFT, padx=1)

        # # Signal label with adjusted properties
        # self.signal_label = tk.Label(
        #     self.signal_frame,
        #     text="From Set Random",
        #     font=('Arial', 16, 'bold'),  # Changed from 50 to 11 to match other labels
        #     bg='#2C3E50',
        #     fg='#2ECC71',  # Green color for signals
        #     width=8,  # Adjusted width to be more proportional
        #     anchor='center'  # Center the text
        # )
        # self.signal_label.pack(pady=2)

    def setup_status_section_for_set_random(self):
        """Setup the status section with process information"""
        # Create process info frame at the top
        self.process_frame_1 = tk.Frame(self.top_bar, bg='#2C3E50')
        self.process_frame_1.pack(side=tk.TOP, fill='x', pady=2)
        
        top_row = tk.Frame(self.process_frame_1, bg='#2C3E50')
        top_row.pack(fill='x', pady=(0, 1))

        bottom_row = tk.Frame(self.process_frame_1, bg='#2C3E50')
        bottom_row.pack(fill='x', pady=(1, 0))

        # Create labels for process information
        self.process_label = tk.Label(
            top_row,
            text="Process label",
            font=('Arial', 15, 'bold'),
            bg='#2C3E50',
            fg='#2ECC71',  # Green color
        )
        self.process_label.pack(side=tk.LEFT, padx=20)

        # Capture count label
        self.capture_count_label = tk.Label(
            bottom_row,
            text="count label",
            font=('Arial', 12),
            bg='#2C3E50',
            fg='white'
        )
        self.capture_count_label.pack(side=tk.LEFT, padx=10)

        # Add remaining captures label
        self.remaining_label = tk.Label(
            bottom_row,
            text="remaining label",
            font=('Arial', 12),
            bg='#2C3E50',
            fg='#F1C40F'  # Yellow color
        )
        self.remaining_label.pack(side=tk.RIGHT, padx=10)

    def setup_main_structure(self):
        """Setup the main window structure"""
        # Configure main window
        self.root.configure(bg='#f0f0f0')

        # Create main container that will hold all components
        self.main_container = tk.Frame(self.root, bg='#f0f0f0')
        self.main_container.pack(expand=True, fill='both')

    def setup_top_bar(self):
        """Setup the top bar with improved layout"""
        # Create main container
        self.bar_container = tk.Frame(self.main_container)
        self.bar_container.pack(fill='x')

        # Top bar frame
        self.top_bar = tk.Frame(self.bar_container, bg='#2C3E50', height=60)
        self.top_bar.pack(fill='x', side=tk.LEFT, expand=True)
        self.top_bar.pack_propagate(False)

        # Logo section (left side)
        self.logo_frame = tk.Frame(self.top_bar, bg='#2C3E50')
        self.logo_frame.pack(side=tk.LEFT, padx=10)

        try:
            import os
            current_dir = os.getcwd()
            logo_path = os.path.join(current_dir, "assets", "logo.png")

            if os.path.exists(logo_path):
                logo_image = Image.open(logo_path)
                logo_image = logo_image.resize(
                    (40, 40), Image.Resampling.LANCZOS)
                self.logo_photo = ImageTk.PhotoImage(logo_image)
                self.logo_label = tk.Label(
                    self.logo_frame,
                    image=self.logo_photo,
                    bg='#2C3E50'
                )
                self.logo_label.pack(side=tk.LEFT, padx=5)
            else:
                raise FileNotFoundError("Logo file not found")
        except Exception as e:
            print(f"Could not load logo: {e}")
            self.logo_label = tk.Label(
                self.logo_frame,
                text="EYE TRACK",
                font=('Arial', 16, 'bold'),
                bg='#2C3E50',
                fg='white'
            )
            self.logo_label.pack(side=tk.LEFT, padx=5)

        # Controls section
        self.controls_frame = tk.Frame(self.top_bar, bg='#2C3E50')
        self.controls_frame.pack(side=tk.LEFT, expand=True, padx=10)

        # Setup control buttons
        self.setup_control_buttons()

        # Toggle button frame
        self.toggle_frame = tk.Frame(self.bar_container, bg='#2C3E50')
        self.toggle_frame.pack(side=tk.RIGHT, fill='y')

        # Toggle button
        self.toggle_button = tk.Button(
            self.toggle_frame,
            text="≡",
            font=('Arial', 16),
            bg='#2C3E50',
            fg='white',
            relief='flat',
            command=self.toggle_bar_visibility,
            width=2,
            height=1
        )
        self.toggle_button.pack(side=tk.RIGHT, padx=2, pady=2)

        # Bind hover effects
        self.toggle_button.bind(
            '<Enter>', lambda e: self.toggle_button.configure(bg='#34495E'))
        self.toggle_button.bind(
            '<Leave>', lambda e: self.toggle_button.configure(bg='#2C3E50'))

    def setup_control_buttons(self):
        """Setup the control buttons with improved layout"""
        # Create a container for input controls
        input_controls = tk.Frame(self.controls_frame, bg='#2C3E50')
        input_controls.pack(side=tk.LEFT)

        # Create left frame for Times/Delay inputs
        left_input_frame = tk.Frame(input_controls, bg='#2C3E50')
        left_input_frame.pack(side=tk.LEFT, padx=5)

        # Random times input (top)
        self.random_times_frame = tk.Frame(left_input_frame, bg='#2C3E50')
        self.random_times_frame.pack(pady=(0, 2))

        self.random_times_label = tk.Label(
            self.random_times_frame,
            text="Times:  ",
            font=('Arial', 12),
            bg='#2C3E50',
            fg='white'
        )
        self.random_times_label.pack(side=tk.LEFT, padx=4)

        self.random_times_entry = ttk.Entry(
            self.random_times_frame,
            width=5
        )
        self.random_times_entry.insert(0, "1")
        self.random_times_entry.pack(side=tk.LEFT, padx=2)

        # Delay input (bottom)
        self.delay_frame = tk.Frame(left_input_frame, bg='#2C3E50')
        self.delay_frame.pack()

        self.delay_label = tk.Label(
            self.delay_frame,
            text="Delay(s):",
            font=('Arial', 12),
            bg='#2C3E50',
            fg='white'
        )
        self.delay_label.pack(side=tk.LEFT, padx=4)

        self.delay_entry = ttk.Entry(
            self.delay_frame,
            width=5
        )
        self.delay_entry.insert(0, "3")
        self.delay_entry.pack(side=tk.LEFT, padx=2)

        # Action buttons with debug wrappers
        def debug_toggle_preview():
            print("Preview toggle clicked, current state:", self.preview_var.get())
            self.toggle_preview()

        def debug_toggle_parameters():
            print("Parameters toggle clicked, current state:", self.parameter_var.get())
            self.toggle_parameters()
            
        def debug_toggle_Mask():
            self.toggle_mask()
            
        def debug_toggle_Draw_headpose():
            self.toggle_Draw_headpose()
            
        def debug_toggle_Draw_Box():
            self.toggle_Draw_Box()

        # Create a container for buttons with two columns
        button_controls = tk.Frame(self.controls_frame, bg='#2C3E50')
        button_controls.pack(side=tk.LEFT, padx=2)

        # Left column frame
        left_column = tk.Frame(button_controls, bg='#2C3E50')
        left_column.pack(side=tk.LEFT, padx=3)

        # Right column frame
        right_column = tk.Frame(button_controls, bg='#2C3E50')
        right_column.pack(side=tk.LEFT, padx=3)

        # Create buttons in left column
        self.set_random_button = ttk.Button(
            left_column,
            text="Set Random",
            command=self.set_random_parameters,
            width=8
        )
        self.set_random_button.pack(pady=(0, 2))  
        
        self.calibrate_button = ttk.Button(
            left_column,
            text="Calibrate",
            command=self.start_calibration_sequence,
            width=8
        )
        self.calibrate_button.pack()

        # Create buttons in right column
        self.random_button = ttk.Button(
            right_column,
            text="Random Dot",
            command=self.draw_random_dot_and_capture,
            width=8
        )
        self.random_button.pack(pady=(0, 2)) 
        
        self.clear_button = ttk.Button(
            right_column,
            text="Clear All",
            command=self.clear_all,
            width=8
        )
        self.clear_button.pack()

        # Create a container for toggle switches with two rows
        toggle_controls = tk.Frame(self.controls_frame, bg='#2C3E50')
        toggle_controls.pack(side=tk.LEFT, padx=5)

        # Style for toggle switches
        toggle_style = {
            'font': ('Arial', 11),
            'bg': '#2C3E50',
            'fg': 'white',
            'selectcolor': '#3498db',  # Light blue when selected
            'activebackground': '#2C3E50',
            'activeforeground': 'white',
            'highlightthickness': 0,
            'bd': 0,
            'relief': 'flat',
            'padx': 5,
            'pady': 5
        }
        
        # Create ttk style for buttons
        style = ttk.Style()
        style.configure('Hover.TButton', 
                    background='#34495E',
                    relief='flat')
        style.map('Hover.TButton',
                background=[('active', '#34495E'), 
                            ('!active', '#2C3E50')])

        # Update all buttons to use the new style
        for button in [self.set_random_button, self.calibrate_button, 
                    self.random_button, self.clear_button]:
            button.configure(style='Hover.TButton')

        # Top row of toggles
        top_row = tk.Frame(toggle_controls, bg='#2C3E50')
        top_row.pack(fill='x', pady=(0, 1))

        # Bottom row of toggles
        bottom_row = tk.Frame(toggle_controls, bg='#2C3E50')
        bottom_row.pack(fill='x', pady=(1, 0))

        # Top row toggles
        self.preview_var = tk.BooleanVar(value=False)
        self.preview_switch = tk.Checkbutton(
            top_row,
            text="Show Preview",
            variable=self.preview_var,
            command=debug_toggle_preview,
            **toggle_style
        )
        self.preview_switch.pack(side=tk.LEFT, padx=9)

        self.parameter_var = tk.BooleanVar(value=False)
        self.parameter_switch = tk.Checkbutton(
            top_row,
            text="Show Parameters",
            variable=self.parameter_var,
            command=debug_toggle_parameters,
            **toggle_style
        )
        self.parameter_switch.pack(side=tk.LEFT, padx=9)

        # Bottom row toggles
        self.mask_var = tk.BooleanVar(value=False)
        self.mask_switch = tk.Checkbutton(
            bottom_row,
            text="Show Mask",
            variable=self.mask_var,
            command=debug_toggle_Mask,
            **toggle_style
        )
        self.mask_switch.pack(side=tk.LEFT, padx=6)

        self.visualization_var = tk.BooleanVar(value=False)
        self.visualization_switch = tk.Checkbutton(
            bottom_row,
            text="Draw Visualization",
            variable=self.visualization_var,
            command=debug_toggle_Draw_headpose,
            **toggle_style
        )
        self.visualization_switch.pack(side=tk.LEFT, padx=6)

        self.bbox_var = tk.BooleanVar(value=False)
        self.bbox_switch = tk.Checkbutton(
            bottom_row,
            text="Show Bounding Box",
            variable=self.bbox_var,
            command=debug_toggle_Draw_Box,
            **toggle_style
        )
        self.bbox_switch.pack(side=tk.LEFT, padx=6)
                
    def setup_main_content(self):
        """Setup the main content area with the white canvas"""
        # Main content container
        self.content_container = tk.Frame(
            self.main_container,
            bg='#f0f0f0',
            padx=10,
            pady=10
        )
        self.content_container.pack(expand=True, fill='both')

        # White canvas
        self.canvas = tk.Canvas(
            self.content_container,
            width=self.original_canvas_width,
            height=self.original_canvas_height,
            bg='white',
            highlightthickness=1,
            highlightbackground='#d0d0d0'
        )
        self.canvas.pack(expand=True, fill='both')

    def setup_preview_section(self):
        """Setup preview section for both regular and temporary preview"""
        # Create main preview container
        self.preview_container = tk.Frame(
            self.root,
            bg='white',
            relief='solid'
        )
        self.preview_container.pack_propagate(False)

        # Create container for both previews
        self.previews_frame = tk.Frame(
            self.preview_container,
            bg='white'
        )
        self.previews_frame.pack(expand=True, fill='both', padx=5, pady=5)

        # Webcam preview frame
        self.webcam_frame = tk.Frame(
            self.previews_frame,
            bg='white',
            width=self.webcam_width,
            height=self.webcam_height
        )
        self.webcam_frame.pack(side=tk.LEFT, padx=5)
        self.webcam_frame.pack_propagate(False)

        self.webcam_title = tk.Label(
            self.webcam_frame,
            text="Webcam Feed",
            font=('Arial', 10, 'bold'),
            bg='white'
        )
        self.webcam_title.pack(pady=2)

        self.webcam_label = tk.Label(
            self.webcam_frame,
            bg='white'
        )
        self.webcam_label.pack(expand=True, fill='both')

        # Screen capture frame
        self.screen_frame = tk.Frame(
            self.previews_frame,
            bg='white',
            width=self.webcam_width,
            height=self.webcam_height
        )
        self.screen_frame.pack(side=tk.LEFT, padx=5)
        self.screen_frame.pack_propagate(False)

        self.screen_title = tk.Label(
            self.screen_frame,
            text="Screen Capture",
            font=('Arial', 10, 'bold'),
            bg='white'
        )
        self.screen_title.pack(pady=2)

        self.screen_label = tk.Label(
            self.screen_frame,
            bg='white'
        )
        self.screen_label.pack(expand=True, fill='both')

    def setup_parameters_section(self):
        """Setup the parameters display section with all save parameters"""
        # Create parameter container
        self.param_container = tk.Frame(
            self.root,
            bg='#2C3E50',
            relief='solid',
            width=600,
            height=self.webcam_height,
            borderwidth=2
        )
        self.param_container.pack(side=tk.RIGHT, fill='both', expand=True)

        # Create scrollable frame
        self.param_canvas = tk.Canvas(
            self.param_container,
            bg='#2C3E50',
            highlightthickness=0
        )
        self.param_scrollbar = ttk.Scrollbar(
            self.param_container,
            orient='vertical',
            command=self.param_canvas.yview
        )

        # Create frame for parameter content
        self.scrollable_frame = tk.Frame(
            self.param_canvas,
            bg='#2C3E50'
        )

        # Configure scrolling
        self.scrollable_frame.bind(
            '<Configure>',
            lambda e: self.param_canvas.configure(
                scrollregion=self.param_canvas.bbox('all'))
        )
        self.param_canvas.create_window(
            (0, 0), window=self.scrollable_frame, anchor='nw')
        self.param_canvas.configure(yscrollcommand=self.param_scrollbar.set)
        self.param_container.pack(side=tk.RIGHT, fill='y', expand=True)
        # Pack scrollbar and canvas
        self.param_scrollbar.pack(side=tk.RIGHT, fill='y')
        self.param_canvas.pack(side=tk.LEFT, fill='both', expand=True)

        # Update canvas dimensions
        self.current_parameters['canvas_width'] = self.canvas.winfo_width()
        self.current_parameters['canvas_height'] = self.canvas.winfo_height()

        # Define parameter sections and their parameters
        parameter_sections = {
            'Resolution Information': [
                ('Screen Resolution',
                 lambda: f"{self.current_parameters.get('screen_resolution_width', '--')} x {self.current_parameters.get('screen_resolution_height', '--')}"),
                ('Webcam Resolution',
                 lambda: f"{self.current_parameters.get('webcam_resolution_width', '--')} x {self.current_parameters.get('webcam_resolution_height', '--')}"),
                ('Preview Size',
                 lambda: f"{self.current_parameters.get('preview_width', '--')} x {self.current_parameters.get('preview_height', '--')}"),
                ('Canvas Size',
                 lambda: f"{self.current_parameters.get('canvas_width', '--')} x {self.current_parameters.get('canvas_height', '--')}")
            ],
            'Gaze and Posture': [
                ('Gaze Direction', lambda: self.current_parameters.get(
                    'gaze_direction', '--')),
                ('Posture', lambda: self.current_parameters.get('posture', '--')),
                ('Eye Right State', lambda: self.current_parameters.get(
                    'right_eye_state', '--')),
                ('Eye Left State', lambda: self.current_parameters.get(
                    'left_eye_state', '--'))
            ],
            'Measurements': [
                ('Face Distance',
                 lambda: f"{self.current_parameters.get('distance', '--')} cm"),
                ('Eye Distance',
                 lambda: f"{self.current_parameters.get('distance_cm_from_eye', '--')} cm"),
                ('Chin Distance',
                 lambda: f"{self.current_parameters.get('chin_depth', '--')} cm"),

                # ('Vertical Ratio', lambda: format_float(
                #     self.current_parameters.get('vertical_ratio', '--'))),
                # ('Horizontal Ratio', lambda: format_float(
                #     self.current_parameters.get('horizontal_ratio', '--')))
            ],
            'Face Information': [
                ('Face Size',
                 lambda: f"W: {self.current_parameters.get('face_size_width', '--')} H: {self.current_parameters.get('face_size_height', '--')}"),
                ('Pitch', 
                lambda: format_angle(self.current_parameters.get('pitch'))),
                ('Yaw', 
                lambda: format_angle(self.current_parameters.get('yaw'))),
                ('Roll', 
                lambda: format_angle(self.current_parameters.get('roll'))),
                ('Face Min Position', lambda: format_position(
                    self.current_parameters.get('face_min_position', (None, None)))),
                ('Face Max Position', lambda: format_position(
                    self.current_parameters.get('face_max_position', (None, None)))),
                ('Face Center', lambda: format_position(
                    self.current_parameters.get('face_center_position', (None, None)))),
                ('Nose Postion', lambda: format_position(
                    self.current_parameters.get('nose_position', (None, None)))),
                ('Mouth Position',
                 lambda: f"left: {self.current_parameters.get('mouth_left_position', (None, None))}, right: {self.current_parameters.get('mouth_right_position', (None, None))}"),
                ('Cheek Position',
                 lambda: f"left: {self.current_parameters.get('cheek_left_position', (None, None))}, right: {self.current_parameters.get('cheek_right_position', (None, None))}"),
                ('Chin Position', lambda: format_position(
                    self.current_parameters.get('chin_position', (None, None))))
            ],
            'Eye Positions': [
                ('Right Eye Pupil',
                 lambda: format_position(self.current_parameters.get('right_eye_position'))),
                ('Left Eye Pupil',
                 lambda: format_position(self.current_parameters.get('left_eye_position'))),
                ('Right Socket Center',
                 lambda: format_position(self.current_parameters.get('eye_socket_right_center'))),
                ('Left Socket Center',
                 lambda: format_position(self.current_parameters.get('eye_socket_left_center'))),
                ('Center Between Eyes',
                 lambda: format_position(self.current_parameters.get('center_between_eye_left_and_right')))
            ],
            'Eye Boxes & State': [
                ('Right Eye Box',
                 lambda: format_box(self.current_parameters.get('right_eye_box'))),
                ('Left Eye Box',
                 lambda: format_box(self.current_parameters.get('left_eye_box'))),
                ('Eye State',
                 lambda: f"left -> {self.current_parameters.get('left_eye_state', '--')}, right -> {self.current_parameters.get('right_eye_state', '--')}"),
                ('Eye Ear',
                 lambda: f"left: {self.current_parameters.get('left_eye_ear', '--')}, right: {self.current_parameters.get('right_eye_ear', '--')}"),
            ]
        }

        # Create parameter labels
        self.parameter_labels = {}

        for section_title, parameters in parameter_sections.items():
            # Create section frame
            section_frame = tk.Frame(
                self.scrollable_frame, bg='#2C3E50', pady=5)
            section_frame.pack(fill='x', padx=20)

            # Section title
            tk.Label(
                section_frame,
                text=section_title,
                font=('Arial', 14, 'bold'),
                fg='#3498DB',
                bg='#2C3E50'
            ).pack(anchor='w')

            # Parameters in this section
            for title, value_func in parameters:
                param_frame = tk.Frame(section_frame, bg='#2C3E50')
                param_frame.pack(fill='x', pady=2)

                # Parameter title
                tk.Label(
                    param_frame,
                    text=f"{title}:",
                    font=('Arial', 12),
                    fg='white',
                    bg='#2C3E50',
                    width=25,
                    anchor='w'
                ).pack(side=tk.LEFT)

                # Parameter value
                value_label = tk.Label(
                    param_frame,
                    text="--",
                    font=('Arial', 10),
                    fg='#AED6F1',
                    bg='#2C3E50',
                    anchor='w'
                )
                value_label.pack(side=tk.LEFT, padx=5)

                self.parameter_labels[title] = (value_label, value_func)

        # def format_float(value):
        #     """Format float values"""
        #     try:
        #         if isinstance(value, (int, float)):
        #             return f"{float(value):.4f}"
        #         return str(value)
        #     except:
        #         return '--'
        def format_angle(value):
            """Format angle values with proper error handling"""
            try:
                if value is None:
                    return '--'
                if isinstance(value, (int, float)):
                    return f"{value:.2f}°"
                return str(value)
            except:
                return '--'

        def format_position(pos):
            """Format position tuples with proper error handling"""
            try:
                if isinstance(pos, (tuple, list)) and len(pos) == 2:
                    x, y = pos
                    if x is not None and y is not None:
                        return f"({x:.1f}, {y:.1f})"
                elif isinstance(pos, str) and pos.startswith('('):
                    # Handle string representation of tuples
                    pos = eval(pos)
                    if isinstance(pos, (tuple, list)) and len(pos) == 2:
                        x, y = pos
                        if x is not None and y is not None:
                            return f"({x:.1f}, {y:.1f})"
                return '(--,--)'
            except:
                return '(--,--)'

        def format_box(box):
            """Format box coordinates"""
            try:
                if isinstance(box, tuple) and len(box) == 2 and all(isinstance(p, tuple) for p in box):
                    # Handle ((min_x, min_y), (max_x, max_y)) format
                    (min_x, min_y), (max_x, max_y) = box
                    return f"({min_x:.1f}, {min_y:.1f}, {max_x:.1f}, {max_y:.1f})"
                elif isinstance(box, (tuple, list)) and len(box) == 4:
                    # Handle (min_x, min_y, max_x, max_y) format
                    return f"({box[0]:.1f}, {box[1]:.1f}, {box[2]:.1f}, {box[3]:.1f})"
                return '(--,--,--,--)'
            except:
                return '(--,--,--,--)'

    def setup_countdown_labels(self):
        """Setup countdown labels with new layout structure"""
        # Countdown label for dot position on canvas
    
        self.dot_countdown_label = tk.Label(
            self.canvas,
            text="",  # Start with empty text
            font=('Arial', 42, 'bold'),
            fg='red',
            bg='white'
        )
        
        
        # Initially the label is not placed - it will be positioned when needed

        # The main countdown label is already created in setup_status_section
        # We don't need to create it again, but we'll ensure it exists
        if not hasattr(self, 'countdown_label'):
            self.countdown_label = tk.Label(
                self.status_container,
                text="",
                font=('Arial', 11),
                bg='#2C3E50',
                fg='white',
                width=15
            )
            self.countdown_label.pack(side=tk.RIGHT, padx=5)

        # Signal label for status messages
        if not hasattr(self, 'signal_label'):
            self.signal_label = tk.Label(
                self.status_container,
                text="",
                font=('Arial', 50, 'bold'),
                bg='#2C3E50',
                fg='#2ECC71',  # Green color for signals
                width=10
            )
            self.signal_label.pack(side=tk.RIGHT, padx=5)
# ---------------------------------------------------------
    def toggle_mask(self):
        """Toggle mask visualization"""
        state = self.isMaskOn.toggle()
        self.tracker.set_IsMaskOn(state)
        self.mask_var.set(state)  # Update checkbox state
        
    def toggle_Draw_headpose(self):
        """Toggle headpose visualization"""
        state = self.isDrawVisualization.toggle()
        self.tracker.set_IsShowHeadpose(state)
        self.visualization_var.set(state)  # Update checkbox state
        
    def toggle_Draw_Box(self):
        """Toggle bounding box visualization"""
        state = self.isDrawBox.toggle()
        self.tracker.set_IsShowBox(state)
        self.bbox_var.set(state)
    def toggle_bar_visibility(self):
        """Toggle the visibility of the top bar"""
        if bool(self.bar_visible) == True:  # If bar is visible
            self.top_bar.pack_forget()
            self.bar_visible.toggle()
            if hasattr(self, 'toggle_button'):
                self.toggle_button.configure(text="☰")  # Show menu icon
        else:
            self.top_bar.pack(fill='x', side=tk.LEFT, expand=True)
            self.bar_visible.toggle()  
            if hasattr(self, 'toggle_button'):
                self.toggle_button.configure(text="×")  # Show close icon
    def toggle_preview(self):
        """Toggle preview visibility with improved positioning"""
        print("Toggle preview called")
        # self.preview_active = self.preview_var.get()
        state = self.isShowCamera.toggle()
        # self.preview_active = state
        self.preview_var.set(state)

        try:
            if self.isShowCamera:
                print("Activating preview")

                # Calculate center position
                window_width = self.root.winfo_width()
                preview_width = self.webcam_width + 20
                x_position = (window_width - preview_width) // 2

                # Position preview container at the top
                self.preview_container.place(
                    x=x_position,
                    y=70,  # Position below the top bar
                    width=preview_width,
                    height=self.webcam_height + 40
                )

                # # Initialize camera if needed
                # if not hasattr(self, 'cap') or not self.cap.isOpened():
                #     self.cap = cv2.VideoCapture(1)
                #     self.cap.set(cv2.CAP_PROP_FRAME_WIDTH, int(
                #         self.cap.get(cv2.CAP_PROP_FRAME_WIDTH)))
                #     self.cap.set(cv2.CAP_PROP_FRAME_HEIGHT, int(
                #         self.cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))

                print("Starting preview updates")
                self.preview_container.lift()
                self.update_preview()
            else:
                print("Deactivating preview")
                if hasattr(self, 'preview_container'):
                    self.preview_container.place_forget()
                # Clear current photo reference
                if hasattr(self, 'current_photo'):
                    self.current_photo = None

        except Exception as e:
            print(f"Error in toggle_preview: {e}")
            import traceback
            traceback.print_exc()

    # ...............................................

    def toggle_parameters(self):
        """Toggle parameters visibility"""
        state = self.isShowParameter.toggle()
        # self.show_parameters = state
        self.parameter_var.set(state)
        # self.show_parameters = self.parameter_var.get()

        try:
            if self.isShowParameter:
                # Calculate positions
                window_width = self.root.winfo_width()
                preview_width = self.webcam_width + 20
                param_width = 400

                if self.isShowCamera:
                    # Position next to preview
                    preview_x = (window_width - preview_width) // 2
                    param_x = preview_x + preview_width + 10
                else:
                    # Center parameters if no preview
                    param_x = (window_width - param_width) // 2

                # Position parameters container
                self.param_container.place(
                    x=param_x,
                    y=70,  # Match preview y-position
                    width=param_width,
                    height=self.webcam_height + 40
                )

                self.param_container.lift()
                self.update_parameters_display()
            else:
                self.param_container.place_forget()

        except Exception as e:
            print(f"Error in toggle_parameters: {e}")
            import traceback
            traceback.print_exc()

    def update_parameters_display(self):
        """Update the parameter values in the display"""
        if not self.isShowParameter:
            return

        try:
            for title, (label, value_func) in self.parameter_labels.items():
                try:
                    value = value_func()
                    label.config(text=str(value))
                except Exception as e:
                    print(f"Error updating parameter {title}: {e}")
                    label.config(text="--")
        except Exception as e:
            print(f"Error in update_parameters_display: {e}")

    # ...............................................

    def update_preview(self):
        """Update preview with proper error handling and initialization"""
        try:
            if not self.isShowCamera:
                return
            ret, frame = self.cap.read()
            if not ret:
                print("Failed to read frame from camera")
                return
            self.current_parameters['webcam_resolution_width'] = int(
                frame.shape[1])
            self.current_parameters['webcam_resolution_height'] = int(
                frame.shape[0])
            self.current_parameters['preview_width'] = self.webcam_width
            self.current_parameters['preview_height'] = self.webcam_height
            # Process webcam frame
            frame_processed = self.process_frame_main(frame.copy())
            frame_resized = cv2.resize(
                frame_processed, (self.webcam_width, self.webcam_height))
            frame_rgb = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2RGB)

            # Create new photo image and keep reference
            self.current_photo = ImageTk.PhotoImage(
                image=Image.fromarray(frame_rgb))

            # Update webcam preview
            if hasattr(self, 'webcam_label') and self.webcam_label.winfo_exists():
                self.webcam_label.configure(image=self.current_photo)
                # print("Preview updated successfully")
            # if hasattr(self, 'screen_label') and self.screen_label.winfo_exists():
            #     # Capture current canvas state
            #     canvas_x = self.canvas.winfo_rootx()
            #     canvas_y = self.canvas.winfo_rooty()
            #     canvas_width = self.canvas.winfo_width()
            #     canvas_height = self.canvas.winfo_height()

            #     screen_capture = pyautogui.screenshot(
            #         region=(canvas_x, canvas_y,
            #                 canvas_width, canvas_height)
            #     )
            #     screen_resized = screen_capture.resize(
            #         (self.webcam_width, self.webcam_height))
            #     screen_photo = ImageTk.PhotoImage(image=screen_resized)
            #     self.screen_label.configure(image=screen_photo)
            #     self.screen_label.image = screen_photo
             # Update parameters if enabled
            if self.isShowParameter:
                self.update_parameters_display()

            # Schedule next update if still active
            if self.isShowCamera:
                self.root.after(10, self.update_preview)

        except Exception as e:
            print(f"Error in update_preview: {e}")
            import traceback
            traceback.print_exc()

    def get_eye_bbox(self, landmarks, eye_points):
        point_array = np.array([landmarks[idx]
                               for group in eye_points.values() for idx in group])
        x_min = int(np.min(point_array[:, 0])) - 10
        x_max = int(np.max(point_array[:, 0])) + 10
        y_min = int(np.min(point_array[:, 1])) - 10
        y_max = int(np.max(point_array[:, 1])) + 10
        return (x_min, y_min), (x_max, y_max)

    def measure_distance(self, face_width_pixels):
        return (self.FACE_WIDTH_CM * self.focal_length) / face_width_pixels

    def update_y_position(self, y_position):
        return y_position + 54

    def process_frame_main(self, frame):
        """
        Process a video frame with face tracking and update all metrics

        Args:
            frame: Video frame from webcam

        Returns:
            Processed frame with annotations
        """
        try:
            # current_time = int(time.time() * 1000)
            # Process frame and get metrics
            metrics, frame = self.tracker.process_frame(frame, self.timestamp_ms)
            # self.tracker.set_visualization(False)

            if metrics:
                # Extract face box coordinates
                face_min, face_max = metrics.face_box

                # Extract landmark positions
                landmarks = metrics.landmark_positions

                # Extract eye boxes
                left_eye_min, left_eye_max = metrics.left_eye_box
                right_eye_min, right_eye_max = metrics.right_eye_box

                # Get eye states and EAR values
                left_eye_state, left_ear = metrics.left_eye_state
                right_eye_state, right_ear = metrics.right_eye_state

                # Get eye centers
                left_center, right_center, between_eyes = metrics.eye_centers
                # print(left_center, right_center, between_eyes)

                # Get iris centers
                eye_iris_left, eye_iris_right = metrics.eye_iris_center

                # Get depth measurements
                face_depth, left_eye_depth, right_eye_depth, chin_depth = metrics.depths

                # Calculate face center
                face_center_x = (face_min[0] + face_max[0]) // 2
                face_center_y = (face_min[1] + face_max[1]) // 2

                # Calculate face dimensions
                face_width = face_max[0] - face_min[0]
                face_height = face_max[1] - face_min[1]
                pitch, yaw, roll = metrics.head_pose_angles

                # Update parameters dictionary
                self.current_parameters.update({
                    'canvas_width': self.canvas.winfo_width(),
                    'canvas_height': self.canvas.winfo_height(),

                    'face_min_position': face_min,
                    'face_max_position': face_max,
                    'face_center_position': (face_center_x, face_center_y),
                    'face_size_width': face_width,
                    'face_size_height': face_height,

                    # Landmark positions
                    'nose_position': landmarks['Nose'],
                    'mouth_left_position': landmarks['Mouth_L'],
                    'mouth_right_position': landmarks['Mouth_R'],
                    'chin_position': landmarks['Chin'],
                    'cheek_left_position': landmarks['Cheek_L'],
                    'cheek_right_position': landmarks['Cheek_R'],

                    # Eye boxes
                    'left_eye_box': (left_eye_min, left_eye_max),
                    'right_eye_box': (right_eye_min, right_eye_max),

                    # Eye states
                    'left_eye_state': left_eye_state,
                    'right_eye_state': right_eye_state,
                    'left_eye_ear': round(left_ear, 3),
                    'right_eye_ear': round(right_ear, 3),

                    # Eye centers
                    'eye_socket_left_center': left_center,
                    'eye_socket_right_center': right_center,
                    'center_between_eye_left_and_right': between_eyes,

                    # Iris centers
                    'left_eye_position': eye_iris_left,
                    'right_eye_position': eye_iris_right,

                    # Depth measurements
                    'distance': round(face_depth, 3),
                    'distance_cm_from_eye': round((left_eye_depth + right_eye_depth) / 2, 3),
                    'chin_depth': round(chin_depth, 3),
                    
                    'pitch': round(pitch, 3),
                    'yaw': round(yaw, 3),
                    'roll': round(roll, 3)
                })
                if hasattr(self, 'distance_label'):
                    self.distance_label.config(
                        text=f"Distance: {round(face_depth, 2)} cm")

                # Update gaze direction based on iris positions
                if eye_iris_left and eye_iris_right:
                    avg_x = (eye_iris_left[0] + eye_iris_right[0]) / 2
                    center_x = between_eyes[0]

                    if abs(avg_x - center_x) < 5:
                        self.current_parameters['gaze_direction'] = "Looking center"
                    elif avg_x < center_x:
                        self.current_parameters['gaze_direction'] = "Looking left"
                    else:
                        self.current_parameters['gaze_direction'] = "Looking right"

                # Update posture based on face dimensions
                aspect_ratio = face_width / face_height if face_height != 0 else 1
                if 0.9 <= aspect_ratio <= 1.1:
                    self.current_parameters['posture'] = "Forward"
                elif aspect_ratio > 1.1:
                    self.current_parameters['posture'] = "Turned Left/Right"
                else:
                    if face_center_y < frame.shape[0] // 2:
                        self.current_parameters['posture'] = "Looking Up"
                    else:
                        self.current_parameters['posture'] = "Looking Down"
                if hasattr(self, 'distance_label'):
                    self.distance_label.config(text=f"Distance: {round(face_depth, 2)} cm")

                if hasattr(self, 'posture_label'):
                    self.posture_label.config(text=f"Posture: {self.current_parameters['posture']}")
                try:
                    if hasattr(self, 'position_left_label') and self.position_left_label:
                        self.position_left_label.config(text=f"Left: {str(eye_iris_left)}")
                    if hasattr(self, 'position_right_label') and self.position_right_label:
                        self.position_right_label.config(text=f"Right: {str(eye_iris_right)}")
                    if hasattr(self, 'gaze_label') and self.gaze_label:
                        self.gaze_label.config(text=f"Gaze: {self.current_parameters['gaze_direction']}")
                except Exception as e:
                    print(f"Warning: Error updating GUI elements - {e}")

                # Update GUI labels
            if self.isShowParameter:
                self.update_parameters_display()
                # if self.show_parameters:
                #     self.tracker.set_visualization(True)
                #     self.update_parameters_display()
                # else:
                #     self.tracker.set_visualization(False)

                # Draw annotations on frame
                # frame = self.draw_annotations(frame)

            self.timestamp_ms += 10
            return frame

        except Exception as e:
            print(f"Error in process_frame: {e}")
            import traceback
            traceback.print_exc()
            return frame

    def set_random_parameters(self):
        """Set random parameters for multiple captures"""
        try:
            num_times = int(self.random_times_entry.get())
            if num_times <= 0:
                print("Please enter a positive number")
                return

            # Disable buttons during capture sequence
            self.set_random_button.config(state=tk.DISABLED)
            self.random_button.config(state=tk.DISABLED)

            # Schedule multiple captures
            self.schedule_multiple_captures(num_times)

        except ValueError:
            print("Please enter a valid number")

    # Modify schedule_multiple_captures to use the scale value

    def schedule_multiple_captures(self, remaining_times):
        if remaining_times > 0:
            try:
                # self.signal_label.config(
                #     text=f"...{remaining_times}...")
                self.process_label.config(
                    text=f"...{remaining_times}..."
                )
                # Get delay from entry in milliseconds
                delay_s = float(self.delay_entry.get())
                if delay_s <= 0:
                    print("Please enter a positive delay time")
                    return

                delay_ms = int(delay_s * 1000)

                def capture_with_delay():
                    self.draw_random_dot_and_capture()
                    # Schedule next capture after current one completes
                    self.root.after(delay_ms,
                                    lambda: self.schedule_multiple_captures(remaining_times - 1))

                self.root.after(5000, capture_with_delay)
            except ValueError:
                print("Please enter a valid delay time")
                # self.signal_label.config(text=f"Error.....")
                self.process_label.config(
                    text=f"Error....."
                )
                self.enable_button()
        else:
            # self.signal_label.config(text=f"- Done -")
            # self.root.after(1000, self.signal_label.config(text=f""))
            self.process_label.config(
                text=f"-- Done --"
            )
            self.root.after(1000, self.process_label.config(text=f""))
            self.enable_button()

    def update_dot(self, random_position=False):
        # Clear previous dots
        if hasattr(self, 'dot_id'):
            self.canvas.delete(self.dot_id)
        if hasattr(self, 'current_dot'):
            self.canvas.delete(self.current_dot)

        # Generate new position if random is requested
        if random_position:
            self.dot_x = random.randint(1, self.canvas.winfo_width()-1)
            self.dot_y = random.randint(1, self.canvas.winfo_height()-1)

        # Update parameters
        self.current_parameters['dot_position_x'] = self.dot_x
        self.current_parameters['dot_position_y'] = self.dot_y

        # Draw new dot
        dot_radius = 5
        self.dot_id = self.canvas.create_oval(
            self.dot_x - dot_radius,
            self.dot_y - dot_radius,
            self.dot_x + dot_radius,
            self.dot_y + dot_radius,
            fill='red',
            outline='red'
        )
        self.current_dot = self.dot_id  # Keep both references synchronized

        # # Update label position if it exists
        # if bool(self.isShowLabelOnDot) == True:
        #     self.update_dot_label_position()
        if hasattr(self, 'dot_countdown_label'):
            self.update_dot_label_position()

    def draw_random_dot_and_capture(self):
        """Handle random dot generation and capture sequence"""
        print("Starting random dot capture sequence")
        try:
            # Hide all UI elements
            self.hide_ui_elements()

            # Reset countdown label state
            self.isShowLabelOnDot = Toggle(True)  # Start with label visible
            
            # Clear any existing countdown label
            if hasattr(self, 'dot_countdown_label'):
                self.dot_countdown_label.config(text="")
                self.dot_countdown_label.place_forget()
            
            # Ensure countdown label is properly initialized
            self.setup_countdown_labels()
                
            # if hasattr(self, 'dot_countdown_label'):
            #     self.dot_countdown_label.place_forget()
            if self.current_dot:
                self.canvas.delete(self.current_dot)
            # Draw new random dot
            self.update_dot(random_position=True)
            # Start countdown
            self.root.after(1, lambda: self.countdown(3))
        except Exception as e:
            print(f"Error in draw_random_dot_and_capture: {e}")

    def hide_ui_elements(self):
        """Hide all UI elements before capture"""
        print("Hiding UI elements...")
        try:
            # Hide top bar
            # self.top_bar.pack_forget()
            if bool(self.bar_visible) == True:
                self.toggle_bar_visibility()
            
            if self.preview_var:
                self.toggle_preview()
            
            if self.isMaskOn:
                self.toggle_mask()
            if self.isDrawBox:
                self.toggle_Draw_Box()
            if self.isDrawVisualization:
                self.toggle_Draw_headpose()
            if self.isShowParameter:
                self.toggle_parameters()
                
            if hasattr(self, 'preview_container'):
                self.preview_container.place_forget()
                
            if hasattr(self, 'param_container'):
                self.param_container.place_forget()

            # Disable buttons during capture
            self.calibrate_button.config(state=tk.DISABLED)
            self.random_button.config(state=tk.DISABLED)
            self.set_random_button.config(state=tk.DISABLED)

        except Exception as e:
            print("Error hiding UI elements:", e)
    def enable_button(self):
        self.calibrate_button.config(state=tk.NORMAL)
        self.random_button.config(state=tk.NORMAL)
        self.set_random_button.config(state=tk.NORMAL)

    def show_temp_preview(self, filename_base, screen_capture):
        """Show temporary preview for both webcam and screen capture"""
        try:
            print("Starting preview display...")
            
            # Calculate preview position
            window_width = self.root.winfo_width()
            preview_width = (self.webcam_width * 2) + 30
            preview_height = self.webcam_height + 50
            x_position = (window_width - preview_width) // 2

            # Ensure preview container exists
            # if self.bar_visible == False:
            #     self.toggle_bar_visibility()
            # if not hasattr(self, 'preview_container') :
            #     # self.setup_preview_section()
            #     self.toggle_bar_visibility()

            # Configure preview container
            self.preview_container.configure(width=preview_width, height=preview_height)
            self.preview_container.place(
                x=x_position,
                y=70,
                width=preview_width,
                height=preview_height
            )

            # Load and display webcam image from saved file
            webcam_path = os.path.join(self.output_dir, f"webcam_{filename_base}.jpg")
            print(f"Loading webcam image from: {webcam_path}")
            
            if os.path.exists(webcam_path):
                # Read the saved webcam image
                webcam_image = Image.open(webcam_path)
                webcam_resized = webcam_image.resize((self.webcam_width, self.webcam_height))
                self.webcam_photo = ImageTk.PhotoImage(image=webcam_resized)
                self.webcam_label.configure(image=self.webcam_photo)
                print("Webcam image loaded and displayed")
            else:
                print(f"Webcam image file not found: {webcam_path}")
            
            # Process and display screen capture
            print("Processing screen capture...")
            screen_resized = screen_capture.resize((self.webcam_width, self.webcam_height))
            self.screen_photo = ImageTk.PhotoImage(image=screen_resized)
            self.screen_label.configure(image=self.screen_photo)
            print("Screen capture processed and displayed")

            # Ensure preview is visible and on top
            self.preview_container.lift()
            print("Preview container lifted and visible")

            # Update the display immediately
            self.preview_container.update_idletasks()
            # self.capture_in_progress = False
            
            # Schedule preview removal
            self.root.after(2000, self.hide_temp_preview)
            
        except Exception as e:
            print(f"Error in show_temp_preview: {e}")
            import traceback
            traceback.print_exc()
            self.restore_ui()

    def restore_ui(self):
        """Restore UI elements to their original state"""
        print("Restoring UI...")
        try:
            # Show top bar
            # self.top_bar.pack(fill='x', side=tk.TOP)
            if bool(self.bar_visible) == False:
                print("Bar hidden on restore ui")
                self.toggle_bar_visibility()

            # Clear any remaining countdown or capture text
            self.countdown_label.config(text="")

            # Enable buttons
            self.enable_button()
            # Reset capture flag
            self.capture_in_progress = False

        except Exception as e:
            print("Error restoring UI:", e)

    def save_parameters_to_csv(self, filename_base):
        """
        Save all parameters to a CSV file with coordinates split into x and y components

        Args:
            filename_base: Base filename for the output CSV
        """
        csv_path = os.path.join(
            self.output_dir, f"parameters_{filename_base}.csv")

        def safe_eval_position(pos_str, default=(None, None)):
            """Safely evaluate position strings to tuples"""
            try:
                if not pos_str or pos_str == 'None':
                    return default
                if isinstance(pos_str, (tuple, list)):
                    return pos_str
                return eval(pos_str)
            except:
                return default

        def format_value(value):
            """Format values consistently"""
            if value is None:
                return "N/A"
            if isinstance(value, float):
                return f"{value:.4f}"
            return str(value)

        try:
            # Update canvas dimensions
            self.current_parameters.update({
                'canvas_width': self.canvas.winfo_width(),
                'canvas_height': self.canvas.winfo_height()
            })

            # Prepare data dictionary
            data = {
                # Metadata
                'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                'file_number': filename_base,

                # Resolution information
                'screen_resolution_width': self.current_parameters.get('screen_resolution_width', 'N/A'),
                'screen_resolution_height': self.current_parameters.get('screen_resolution_height', 'N/A'),
                'webcam_resolution_width': self.current_parameters.get('webcam_resolution_width', 'N/A'),
                'webcam_resolution_height': self.current_parameters.get('webcam_resolution_height', 'N/A'),
                'preview_width': self.current_parameters.get('preview_width', 'N/A'),
                'preview_height': self.current_parameters.get('preview_height', 'N/A'),
                'canvas_width': self.current_parameters.get('canvas_width', 'N/A'),
                'canvas_height': self.current_parameters.get('canvas_height', 'N/A'),

                # Status information
                'gaze_direction': self.current_parameters.get('gaze_direction', 'N/A'),
                'posture': self.current_parameters.get('posture', 'N/A'),

                # Eye states and measurements
                'left_eye_state': self.current_parameters.get('left_eye_state', 'N/A'),
                'right_eye_state': self.current_parameters.get('right_eye_state', 'N/A'),
                'left_eye_ear': format_value(self.current_parameters.get('left_eye_ear')),
                'right_eye_ear': format_value(self.current_parameters.get('right_eye_ear')),

                # Distance measurements
                'distance_cm_from_face': format_value(self.current_parameters.get('distance')),
                'distance_cm_from_eye': format_value(self.current_parameters.get('distance_cm_from_eye')),
                'chin_depth': format_value(self.current_parameters.get('chin_depth')),

                # Face measurements
                'face_size_width': format_value(self.current_parameters.get('face_size_width')),
                'face_size_height': format_value(self.current_parameters.get('face_size_height')),
                
                'pitch': format_value(self.current_parameters.get('pitch')),
                'yaw': format_value(self.current_parameters.get('yaw')),
                'roll': format_value(self.current_parameters.get('roll')),
                
            }

            # Process dot position
            dot_pos = {
                'x': self.current_parameters.get('dot_position_x'),
                'y': self.current_parameters.get('dot_position_y')
            }
            data.update({
                'dot_position_x': format_value(dot_pos['x']),
                'dot_position_y': format_value(dot_pos['y'])
            })

            # Process all position pairs
            position_pairs = {
                'face_min_position': safe_eval_position(self.current_parameters.get('face_min_position')),
                'face_max_position': safe_eval_position(self.current_parameters.get('face_max_position')),
                'face_center_position': safe_eval_position(self.current_parameters.get('face_center_position')),
                'nose_position': safe_eval_position(self.current_parameters.get('nose_position')),
                'chin_position': safe_eval_position(self.current_parameters.get('chin_position')),
                'mouth_left_position': safe_eval_position(self.current_parameters.get('mouth_left_position')),
                'mouth_right_position': safe_eval_position(self.current_parameters.get('mouth_right_position')),
                'cheek_left_position': safe_eval_position(self.current_parameters.get('cheek_left_position')),
                'cheek_right_position': safe_eval_position(self.current_parameters.get('cheek_right_position')),
                'left_eye_position': safe_eval_position(self.current_parameters.get('left_eye_position')),
                'right_eye_position': safe_eval_position(self.current_parameters.get('right_eye_position')),
                'eye_socket_left_center': safe_eval_position(self.current_parameters.get('eye_socket_left_center')),
                'eye_socket_right_center': safe_eval_position(self.current_parameters.get('eye_socket_right_center')),
                'center_between_eyes': safe_eval_position(self.current_parameters.get('center_between_eye_left_and_right'))
            }

            # Add all position pairs to data dictionary as separate x/y components
            for key, (x, y) in position_pairs.items():
                data.update({
                    f'{key}_x': format_value(x),
                    f'{key}_y': format_value(y)
                })

            # Process eye boxes
            eye_boxes = {
                'left_eye_box': self.current_parameters.get('left_eye_box'),
                'right_eye_box': self.current_parameters.get('right_eye_box')
            }

            # Add eye box coordinates
            for eye, box in eye_boxes.items():
                try:
                    if isinstance(box, tuple) and len(box) == 2 and all(isinstance(p, tuple) for p in box):
                        # Handle ((min_x, min_y), (max_x, max_y)) format
                        (min_x, min_y), (max_x, max_y) = box
                    elif isinstance(box, (tuple, list)) and len(box) == 4:
                        # Handle (min_x, min_y, max_x, max_y) format
                        min_x, min_y, max_x, max_y = box
                    else:
                        min_x = min_y = max_x = max_y = None
                except:
                    min_x = min_y = max_x = max_y = None

                data.update({
                    f'{eye}_min_x': format_value(min_x),
                    f'{eye}_min_y': format_value(min_y),
                    f'{eye}_max_x': format_value(max_x),
                    f'{eye}_max_y': format_value(max_y)
                })

            # Write to CSV
            with open(csv_path, 'w', newline='') as f:
                writer = csv.writer(f)
                writer.writerow(['Parameter', 'Value'])  # Header
                for key, value in sorted(data.items()):
                    writer.writerow([key, value])

            print(f"Successfully saved parameters to {csv_path}")

            # # Debug output
            # print("\nSaved parameters by section:")
            # sections = {
            #     "Metadata": ['timestamp', 'file_number'],
            #     "Screen and Canvas": ['screen_resolution_width', 'screen_resolution_height',
            #                           'canvas_width', 'canvas_height'],
            #     "Target": ['dot_position_x', 'dot_position_y'],
            #     "Face Measurements": ['face_size_width', 'face_size_height',
            #                           'face_min_position_x', 'face_min_position_y',
            #                           'face_max_position_x', 'face_max_position_y',
            #                           'face_center_position_x', 'face_center_position_y'],
            #     "Eye States": ['left_eye_state', 'right_eye_state',
            #                    'left_eye_ear', 'right_eye_ear'],
            #     "Eye Positions": ['left_eye_position_x', 'left_eye_position_y',
            #                       'right_eye_position_x', 'right_eye_position_y'],
            #     "Distances": ['distance_cm_from_face', 'distance_cm_from_eye', 'chin_depth']
            # }

            # for section, keys in sections.items():
            #     print(f"\n{section}:")
            #     for key in keys:
            #         if key in data:
            #             print(f"  {key}: {data[key]}")

        except Exception as e:
            print(f"Error saving parameters: {e}")
            import traceback
            traceback.print_exc()
            
    def get_calibration_points(self):
        width = self.canvas.winfo_width()
        height = self.canvas.winfo_height()
        
        def conditional_round(height, percetage):
            result = (height * percetage)
            check = (result * 10) % 2 == 0
            if check:
                return round(result)
            return int(result)
        
        first_frame_percetage = 0.12
        second_frame_percetage = 0.26
        # frist frame
        x_left_first = conditional_round(width, first_frame_percetage)
        x_right_first = width - conditional_round(width, first_frame_percetage)
        
        y_top_frist = conditional_round(height, first_frame_percetage)
        y_bottom_frist = height - conditional_round(height, first_frame_percetage)
    
        #second frame
        x_left_second = conditional_round(width, second_frame_percetage)
        x_right_second = width - conditional_round(width, second_frame_percetage)
        
        y_top_second = conditional_round(height, second_frame_percetage)
        y_bottom_second = height - conditional_round(height, second_frame_percetage)
        
        return [
            # first frame
            (x_left_first, y_top_frist), # dot1
            (width // 2, y_top_frist), # dot2
            (x_right_first, y_top_frist), # dot3
            (x_left_first, height//2), # dot4
            (x_right_first, height//2), # dot5
            (x_left_first, y_bottom_frist), # dot6
            (width // 2, y_bottom_frist), # dot7
            (x_right_first, y_bottom_frist), # dot8
            # second frame
            (x_left_second, y_top_second), # dot9
            (width//2, y_top_second), # dot10
            (x_right_second, y_top_second), # dot11
            (x_left_second, height//2), # dot12
            (x_right_second, height//2), # dot13
            (x_left_second, y_bottom_second), # dot14
            (width // 2, y_bottom_second), # dot15
            (x_right_second, y_bottom_second) # dot16
        ]
    def start_calibration_sequence(self):
        """Initialize and start the calibration sequence"""
        try:
            # Hide UI elements
            self.hide_ui_elements()

            # Get calibration points
            self.calibration_points = self.get_calibration_points()
            self.current_calibration_index = 0

            # Get first point
            point = self.calibration_points[self.current_calibration_index]
            self.dot_x, self.dot_y = point

            # Update dot position
            self.update_dot(random_position=False)

            # Update progress in UI
            remaining = len(self.calibration_points) - self.current_calibration_index
            self.process_label.config(
                text=f"Calibration Progress: {self.current_calibration_index + 1}/{len(self.calibration_points)}"
            )
            self.remaining_label.config(
                text=f"Remaining: {remaining}"
            )

            # Reset countdown label state
            self.isShowLabelOnDot = Toggle(True)  # Start with label visible
            
            # Clear any existing countdown label
            if hasattr(self, 'dot_countdown_label'):
                self.dot_countdown_label.config(text="")
                self.dot_countdown_label.place_forget()
            
            # Ensure countdown label is properly initialized
            self.setup_countdown_labels()

            # Start countdown for first point
            # self.root.after(1000, lambda: self.calibration_countdown(3))
            self.root.after(1000, lambda: self.countdown(3, switch_set=True))
                
        except Exception as e:
            print(f"Error starting calibration: {e}")
            self.restore_ui()
    def move_to_next_calibration_point(self):
        """Move to the next calibration point after preview"""
        try:
            # Hide UI elements
            self.hide_ui_elements()
            
            # Move to next point
            self.current_calibration_index += 1
            
            # Check if we're done
            if self.current_calibration_index >= len(self.calibration_points):
                self.finish_calibration()
                return
                
            # Get next point
            point = self.calibration_points[self.current_calibration_index]
            self.dot_x, self.dot_y = point
            
            # Update dot on screen
            self.update_dot(random_position=False)
            
            # Update progress indicators
            remaining = len(self.calibration_points) - self.current_calibration_index
            self.process_label.config(
                text=f"Calibration Progress: {self.current_calibration_index + 1}/{len(self.calibration_points)}"
            )
            self.remaining_label.config(
                text=f"Remaining: {remaining}"
            )
        
            self.countdown(3, switch_set=True)
            
        except Exception as e:
            print(f"Error moving to next calibration point: {e}")
            self.restore_ui()
    def finish_calibration(self):
        """Clean up after calibration sequence"""
        try:
            # Clear calibration-specific attributes
            if hasattr(self, 'calibration_points'):
                del self.calibration_points
            if hasattr(self, 'current_calibration_index'):
                del self.current_calibration_index
                
            # Clear progress displays
            self.process_label.config(text="")
            self.remaining_label.config(text="")
            
            # Restore UI
            self.restore_ui()
            self.current_calibration_index = 0
            
            self.enable_button()
            
            # Clear any remaining dots
            self.clear_all()
            
            print("Calibration sequence completed successfully")
            
        except Exception as e:
            print(f"Error finishing calibration: {e}")
            self.restore_ui()

    def update_dot_label_position(self):
        # Get label dimensions
        label_width = self.dot_countdown_label.winfo_reqwidth()
        label_height = self.dot_countdown_label.winfo_reqheight()

        # Get screen/canvas bounds
        canvas_width = self.canvas.winfo_width()
        canvas_height = self.canvas.winfo_height()

        # Default position (centered above dot)
        x = self.dot_x - label_width//2
        y = self.dot_y - 50

        # Check horizontal bounds
        if x < 0:  # Too far left
            x = 0
        elif x + label_width > canvas_width:  # Too far right
            x = canvas_width - label_width

        # Check vertical bounds
        if y < 0:  # Too high
            y = 0
        elif y + label_height > canvas_height:  # Too low
            y = canvas_height - label_height

        # Handle corner cases
        if x < 0 and y < 0:  # Top-left corner
            x = self.dot_x + 20
            y = self.dot_y + 20
        elif x + label_width > canvas_width and y < 0:  # Top-right corner
            x = self.dot_x - label_width - 20
            y = self.dot_y + 20
        elif x < 0 and y + label_height > canvas_height:  # Bottom-left corner
            x = self.dot_x + 20
            y = self.dot_y - label_height - 20
        elif x + label_width > canvas_width and y + label_height > canvas_height:  # Bottom-right corner
            x = self.dot_x - label_width - 20
            y = self.dot_y - label_height - 20
        return x, y

    def countdown(self, count, switch_set=False):
        if count > 0:
            self.capture_in_progress = True
            countdown_text = str(count)
            
            # Update both countdown labels
            if hasattr(self, 'countdown_label'):
                self.countdown_label.config(text=countdown_text)

            # if hasattr(self, 'dot_countdown_label'):
            if hasattr(self, 'dot_countdown_label'):
                self.dot_countdown_label.config(text=countdown_text)
                self.dot_countdown_label.update_idletasks()

                # Only place the dot countdown label if we have a current dot
                if hasattr(self, 'dot_x') and hasattr(self, 'dot_y'):
                    x_label, y_label = self.update_dot_label_position()
                    self.dot_countdown_label.place(x=x_label, y=y_label)
                    self.dot_countdown_label.lift()
            self.root.after(800, lambda: self.countdown(count - 1, switch_set))
        else:
            if hasattr(self, 'countdown_label'):
                self.countdown_label.config(text="Capturing...")
            # Clear and hide dot countdown label before capture
            if hasattr(self, 'dot_countdown_label'):
                self.dot_countdown_label.update_idletasks()
                self.dot_countdown_label.config(text="")  
                self.dot_countdown_label.place_forget() 
                
            # Toggle the label state and clean up
            self.isShowLabelOnDot = Toggle(False)
                
            # Do the capture
            self.root.after(100, self.capture_sequence())
            # Schedule after-capture actions
            if switch_set:
                # Move to next calibration point
                self.root.after(3000, self.move_to_next_calibration_point)

    def update_next_available_number(self):
        """Find the next available file number by checking existing files"""
        while True:
            webcam_file = os.path.join(
                self.output_dir,
                f"webcam_{self.file_counter:03d}.jpg"
            )
            screen_file = os.path.join(
                self.output_dir,
                f"screen_{self.file_counter:03d}.jpg"
            )

            if not os.path.exists(webcam_file) and not os.path.exists(screen_file):
                break
            self.file_counter += 1

    def capture_sequence(self):
        # Generate filename with current counter
        try:
            if hasattr(self, 'countdown_label'):
                self.countdown_label.config(text="Capturing...")

            # Ensure dot countdown label is cleared and hidden
            if hasattr(self, 'dot_countdown_label'):
                self.dot_countdown_label.config(text="")
                self.dot_countdown_label.place_forget()
                
            filename_base = f"{self.file_counter:03d}"
            webcam_path = os.path.join(
                self.output_dir, f"webcam_{filename_base}.jpg")
            screen_path = os.path.join(
                self.output_dir, f"screen_{filename_base}.jpg")

            ret, frame = self.cap.read()
            if ret:
                # Process frame
                frame_processed = self.process_frame_main(frame.copy())

                # Capture screen
                canvas_x = self.canvas.winfo_rootx()
                canvas_y = self.canvas.winfo_rooty()
                canvas_width = self.canvas.winfo_width()
                canvas_height = self.canvas.winfo_height()

                screen_capture = pyautogui.screenshot(
                    region=(canvas_x, canvas_y, canvas_width, canvas_height)
                )
                screen_capture_rgb = screen_capture.convert('RGB')
                
                # preview_frame = cv2.cvtColor(frame_processed, cv2.COLOR_BGR2RGB)
                # Save captures
                webcam_path = os.path.join(
                    self.output_dir, f"webcam_{filename_base}.jpg")
                screen_path = os.path.join(
                    self.output_dir, f"screen_{filename_base}.jpg")

                # # Save webcam image
                cv2.imwrite(webcam_path, frame_processed)
                screen_capture_rgb.save(screen_path, 'JPEG')
                self.save_parameters_to_csv(filename_base)
                print(f"Files saved successfully: {filename_base}")
                
                # Then show the preview with the current filename_base
                self.root.after(10, lambda: self.show_temp_preview(filename_base, screen_capture))
                    
                    # Update counter after successful capture
                self.file_counter += 1
                self.update_next_available_number()

            else:
                print("Failed to capture frame")

        except Exception as e:
            print(f"Error in capture_sequence:", e)
            import traceback
            traceback.print_exc()
            self.restore_ui()
    def hide_temp_preview(self):
        """Hide temporary preview and restore UI"""
        try:
            # print("Hiding capture preview...")
            
            # Hide container
            # if hasattr(self, 'preview_container'):
            #     self.preview_container.place_forget()
            if self.isShowCamera:
                self.toggle_preview()
            
            # Clean up photo references
            if hasattr(self, 'webcam_photo'):
                self.webcam_photo = None
            if hasattr(self, 'screen_photo'):
                self.screen_photo = None
            
            # Clear labels
            if hasattr(self, 'webcam_label'):
                self.webcam_label.configure(image='')
            if hasattr(self, 'screen_label'):
                self.screen_label.configure(image='')
                
            print("Preview cleanup complete")
            
            # Restore UI
            self.restore_ui()
            
        except Exception as e:
            import traceback
            print(f"Error in hide_capture_preview: {e}")
            traceback.print_exc()
            self.restore_ui()

    def clear_all(self):
        self.canvas.delete("all")
        self.current_dot = None
        self.screen_label.configure(image='')
        self.countdown_label.config(text="")
        self.dot_countdown_label.place_forget()
        self.distance_label.config(text="Distance: -- cm")
        self.posture_label.config(text="Posture: --")
        
        self.enable_button()

    def __del__(self):
        if self.cap.isOpened():
            self.cap.release()


def main():
    root = tk.Tk()
    app = EyeHeadTrackingApp(root)
    root.mainloop()


if __name__ == "__main__":
    main()

I0000 00:00:1745295289.756996 2100671 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M1 Max
W0000 00:00:1745295289.757335 2100671 face_landmarker_graph.cc:174] Sets FaceBlendshapesGraph acceleration to xnnpack by default.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1745295289.761862 2102652 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1745295289.769993 2102652 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
I0000 00:00:1745295289.830017 2100671 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M1 Max
W0000 00:00:1745295289.830261 2100671 face_landmarker_graph.cc:174] Sets FaceBlendshapesGraph acceleration to xnnpack by default.
W0000 00:00:1745295289.834602 2102677 inference_feedback_manager.cc:114] Feedback manage

Using device: mps
Could not load logo: Logo file not found


2025-04-22 11:14:50.056 python[4406:2100671] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.


Preview toggle clicked, current state: True
Toggle preview called
Activating preview
Starting preview updates
Error in process_frame: not enough values to unpack (expected 5, got 2)
Error in process_frame: Input timestamp must be monotonically increasing.
Error in process_frame: Input timestamp must be monotonically increasing.
Error in process_frame: Input timestamp must be monotonically increasing.
Error in process_frame: Input timestamp must be monotonically increasing.


Traceback (most recent call last):
  File "/var/folders/jb/ggq229g94ql147w8sftb9qs80000gn/T/ipykernel_4406/1107242423.py", line 1196, in process_frame_main
    metrics, frame = self.tracker.process_frame(frame, self.timestamp_ms)
  File "/Users/porchportal2/Desktop/🔥everything/eyeTrackingProject/Program/📌Face_Head/Main_model/showframeVisualization.py", line 350, in process_frame
    all_element, (pitch, yaw, roll), (frame_width, frame_height), face_landmarks, frame = self.set_functional_process(frame, timestamp_ms, isVideo)
ValueError: not enough values to unpack (expected 5, got 2)
Traceback (most recent call last):
  File "/var/folders/jb/ggq229g94ql147w8sftb9qs80000gn/T/ipykernel_4406/1107242423.py", line 1196, in process_frame_main
    metrics, frame = self.tracker.process_frame(frame, self.timestamp_ms)
  File "/Users/porchportal2/Desktop/🔥everything/eyeTrackingProject/Program/📌Face_Head/Main_model/showframeVisualization.py", line 350, in process_frame
    all_element, (pitch, yaw

...

Traceback (most recent call last):
  File "/var/folders/jb/ggq229g94ql147w8sftb9qs80000gn/T/ipykernel_4406/1107242423.py", line 1196, in process_frame_main
    metrics, frame = self.tracker.process_frame(frame, self.timestamp_ms)
  File "/Users/porchportal2/Desktop/🔥everything/eyeTrackingProject/Program/📌Face_Head/Main_model/showframeVisualization.py", line 350, in process_frame
    all_element, (pitch, yaw, roll), (frame_width, frame_height), face_landmarks, frame = self.set_functional_process(frame, timestamp_ms, isVideo)
  File "/Users/porchportal2/Desktop/🔥everything/eyeTrackingProject/Program/📌Face_Head/Main_model/showframeVisualization.py", line 321, in set_functional_process
    detection_result = self.detector.detect_for_video(mp_image, timestamp_ms)
  File "/Users/porchportal2/miniforge3/envs/eyeTracking/lib/python3.10/site-packages/mediapipe/tasks/python/vision/face_landmarker.py", line 3184, in detect_for_video
    output_packets = self._process_video_data({
  File "/Users/p

Error in process_frame: Input timestamp must be monotonically increasing.
Error in process_frame: Input timestamp must be monotonically increasing.
Error in process_frame: Input timestamp must be monotonically increasing.
Error in process_frame: Input timestamp must be monotonically increasing.
Error in process_frame: Input timestamp must be monotonically increasing.
Error in process_frame: Input timestamp must be monotonically increasing.
Error in process_frame: Input timestamp must be monotonically increasing.


Traceback (most recent call last):
  File "/var/folders/jb/ggq229g94ql147w8sftb9qs80000gn/T/ipykernel_4406/1107242423.py", line 1196, in process_frame_main
    metrics, frame = self.tracker.process_frame(frame, self.timestamp_ms)
  File "/Users/porchportal2/Desktop/🔥everything/eyeTrackingProject/Program/📌Face_Head/Main_model/showframeVisualization.py", line 350, in process_frame
    all_element, (pitch, yaw, roll), (frame_width, frame_height), face_landmarks, frame = self.set_functional_process(frame, timestamp_ms, isVideo)
  File "/Users/porchportal2/Desktop/🔥everything/eyeTrackingProject/Program/📌Face_Head/Main_model/showframeVisualization.py", line 321, in set_functional_process
    detection_result = self.detector.detect_for_video(mp_image, timestamp_ms)
  File "/Users/porchportal2/miniforge3/envs/eyeTracking/lib/python3.10/site-packages/mediapipe/tasks/python/vision/face_landmarker.py", line 3184, in detect_for_video
    output_packets = self._process_video_data({
  File "/Users/p

Error in process_frame: Input timestamp must be monotonically increasing.
Error in process_frame: Input timestamp must be monotonically increasing.
Error in process_frame: Input timestamp must be monotonically increasing.
Error in process_frame: Input timestamp must be monotonically increasing.
Error in process_frame: Input timestamp must be monotonically increasing.
Error in process_frame: Input timestamp must be monotonically increasing.


Traceback (most recent call last):
  File "/var/folders/jb/ggq229g94ql147w8sftb9qs80000gn/T/ipykernel_4406/1107242423.py", line 1196, in process_frame_main
    metrics, frame = self.tracker.process_frame(frame, self.timestamp_ms)
  File "/Users/porchportal2/Desktop/🔥everything/eyeTrackingProject/Program/📌Face_Head/Main_model/showframeVisualization.py", line 350, in process_frame
    all_element, (pitch, yaw, roll), (frame_width, frame_height), face_landmarks, frame = self.set_functional_process(frame, timestamp_ms, isVideo)
  File "/Users/porchportal2/Desktop/🔥everything/eyeTrackingProject/Program/📌Face_Head/Main_model/showframeVisualization.py", line 321, in set_functional_process
    detection_result = self.detector.detect_for_video(mp_image, timestamp_ms)
  File "/Users/porchportal2/miniforge3/envs/eyeTracking/lib/python3.10/site-packages/mediapipe/tasks/python/vision/face_landmarker.py", line 3184, in detect_for_video
    output_packets = self._process_video_data({
  File "/Users/p

In [2]:
pip install pyautogui


  Using cached PyAutoGUI-0.9.54.tar.gz (61 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached PyMsgBox-1.0.9.tar.gz (18 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pytweening-1.2.0.tar.gz (171 kB)
  Preparing metadata (setup.py) ... done
  Using cached pyscreeze-1.0.1.tar.gz (27 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached PyGetWindow-0.0.9.tar.gz (9.7 kB)
  Preparing metadata (setup.py) ... done
  Using cached MouseInfo-0.1.3.tar.gz (10 kB)
  Preparing metadata (setup.py) ... done
  Using cached PyRect-0.2.0.tar.gz (17 kB)
  Preparing metadata (setup.py) ... done
  Using cached pyperclip-1.9.0.tar.gz (20 kB)
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━

In [9]:
def conditional_round(height):
    result = (height * 0.1)
    check = (result * 10) % 2 == 0
    if check:
        return check, round(result)
    return check, result

# Example usage:
heights = [15, 13, 20, 2, 4, 25, 30, 35, 40, 45, 50]  
for h in heights:
    check, result = conditional_round(h)
    print(f"Height: {h}, Adjusted: {check}, Rounded: {result} -> {int(result)}")

Height: 15, Adjusted: False, Rounded: 1.5 -> 1
Height: 13, Adjusted: False, Rounded: 1.3 -> 1
Height: 20, Adjusted: True, Rounded: 2 -> 2
Height: 2, Adjusted: True, Rounded: 0 -> 0
Height: 4, Adjusted: True, Rounded: 0 -> 0
Height: 25, Adjusted: False, Rounded: 2.5 -> 2
Height: 30, Adjusted: True, Rounded: 3 -> 3
Height: 35, Adjusted: False, Rounded: 3.5 -> 3
Height: 40, Adjusted: True, Rounded: 4 -> 4
Height: 45, Adjusted: False, Rounded: 4.5 -> 4
Height: 50, Adjusted: True, Rounded: 5 -> 5
